In [245]:
import numpy as np 
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
import random
from sklearn.cluster import KMeans

In [246]:
training_data = pd.read_csv("training_data.csv")
training_data = training_data.dropna(subset = ['exit_velocity','distance','hang_time','direction','play_result'])
batters = training_data.groupby("batter_name", as_index=False)
batter_dict = dict(iter(batters))
Seymour_Bobby = batter_dict['Seymour, Bobby']
Seymour_Bobby = Seymour_Bobby[Seymour_Bobby['traditional_cluster'] != 3] #Drop all to 1st
Seymour_Bobby = Seymour_Bobby.reset_index(drop=True)
data = pd.DataFrame(Seymour_Bobby[['distance','direction']])
data = data.reset_index(drop=True)


In [247]:
fielders = pd.DataFrame()
# distances = [110,150,116,146,297,321,295]
# directions = [34,13,-30,-12,-27,1,27]
distances = [110]
directions = [34]
fielders['distance'] = distances
fielders['direction'] = directions
fielders

,distance,direction
0,110,34


In [248]:
data

,distance,direction
0,210.152763,-19.519328
1,4.035207,16.767122
2,3.935966,10.970422
3,309.847859,-8.833638
4,287.297315,-18.036043
...,...,...
245,293.229568,2.609174
246,5.780203,17.874337
247,27.044629,-0.923234
248,249.350455,-23.654236


In [249]:
dfs = [fielders, data]
data_fielders = pd.concat(dfs).reset_index(drop=True)

In [250]:
data_fielders.head(7)

,distance,direction
0,110.000000,34.000000
1,210.152763,-19.519328
2,4.035207,16.767122
3,3.935966,10.970422
4,309.847859,-8.833638
5,287.297315,-18.036043
6,170.318694,17.312409


In [251]:
data_scaled = preprocessing.scale(data)

In [252]:
print(np.array([110,34]))

[110  34]


In [253]:
class K_Means:
    def __init__(self, k,tol=0.0,max_iter = 300):
        self.k = k
        self.tol = tol
        self.max_iter = max_iter
        
    def fit(self,data):
        self.centroids = {}
#         self.centroids[0] = np.array([-0.18521355 , 1.38671137])
        for i in range(self.k):
            self.centroids[i] = data[i] # Adjust first centroid around 1st Base?
            print("Centroid " + str(i) + ": " + str(self.centroids[i]))
            
        for i in range(self.max_iter):
            print("Iteration: " + str(i))
            self.classifications = {}
            
            for i in range(self.k):
                self.classifications[i] = []
                
            for featureset in data:
                distances = [np.linalg.norm(featureset - self.centroids[centroid]) for centroid in self.centroids]
                classification = distances.index(min(distances))
                self.classifications[classification].append(featureset)
                
            prev_centroids = dict(self.centroids)
            
            for classification in self.classifications:
                self.centroids[classification] = np.average(self.classifications[classification], axis = 0)
            
            optimized = True
            
            for c in self.centroids:
                original_centroid = prev_centroids[c]
                current_centroid = self.centroids[c]
                if np.sum((current_centroid - original_centroid)/ original_centroid * 100.0) > self.tol:
                    optimized = False
                
            if optimized:
                break
                
    def predict(self,data):
        distances = [np.linalg.norm(data - self.centroids[centroid]) for centroid in self.centroids]
        classification = distances.index(min(distances))
        return classification

In [254]:
kmeans = KMeans(n_clusters = 6)
groups = kmeans.fit_predict(data_scaled)
Seymour_Bobby['scikit_cluster'] = groups

In [255]:
clf = K_Means(k=6)

In [256]:
clf.fit(data_scaled)

Centroid 0: [ 0.50277333 -1.31658769]
Centroid 1: [-1.18519781  0.82804113]
Centroid 2: [-1.18601053  0.48544025]
Centroid 3: [ 1.3192125  -0.68503409]
Centroid 4: [ 1.13453794 -1.22892146]
Centroid 5: [0.17655775 0.86026912]
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4


In [257]:
len(data)

250

In [258]:
len(Seymour_Bobby)

250

In [259]:
data_scaled[0]

array([ 0.50277333, -1.31658769])

In [260]:
for i, bip in data.iterrows():
    print(i)
    Seymour_Bobby.at[i,'custom_cluster'] = clf.predict(data_scaled[i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249


In [261]:
Seymour_Bobby['custom_cluster'].value_counts()

2.0    60
5.0    45
3.0    43
1.0    42
4.0    33
0.0    27
Name: custom_cluster, dtype: int64

In [262]:
cluster_0 = Seymour_Bobby.loc[Seymour_Bobby['custom_cluster'] == 0]
cluster_1 = Seymour_Bobby.loc[Seymour_Bobby['custom_cluster'] == 1]
cluster_2 = Seymour_Bobby.loc[Seymour_Bobby['custom_cluster'] == 2]
cluster_3 = Seymour_Bobby.loc[Seymour_Bobby['custom_cluster'] == 3]
cluster_4 = Seymour_Bobby.loc[Seymour_Bobby['custom_cluster'] == 4]
cluster_5 = Seymour_Bobby.loc[Seymour_Bobby['custom_cluster'] == 5]
# cluster_6 = Seymour_Bobby.loc[Seymour_Bobby['custom_cluster'] == 6]


custom_clusters = ['cluster_0','cluster_1','cluster_2','cluster_3','cluster_4','cluster_5','cluster_6']

In [263]:
mean_distances = []
mean_distances.append(cluster_0['distance'].mean())
mean_distances.append(cluster_1['distance'].mean())
mean_distances.append(cluster_2['distance'].mean())
mean_distances.append(cluster_3['distance'].mean())
mean_distances.append(cluster_4['distance'].mean())
mean_distances.append(cluster_5['distance'].mean())
mean_distances.append(110)
# mean_distances.append(cluster_6['distance'].mean())

In [264]:
mean_directions = []
mean_directions.append(cluster_0['direction'].mean())
mean_directions.append(cluster_1['direction'].mean())
mean_directions.append(cluster_2['direction'].mean())
mean_directions.append(cluster_3['direction'].mean())
mean_directions.append(cluster_4['direction'].mean())
mean_directions.append(cluster_5['direction'].mean())
mean_directions.append(34)
# mean_directions.append(cluster_6['direction'].mean())

In [265]:
custom_cluster_means = pd.DataFrame()
custom_cluster_means['custom_cluster'] = custom_clusters
custom_cluster_means['distance'] = mean_distances
custom_cluster_means['direction'] = mean_directions

In [266]:
custom_cluster_means

,custom_cluster,distance,direction
0,cluster_0,42.402615,-23.479878
1,cluster_1,36.987707,17.020842
2,cluster_2,44.469809,2.190417
3,cluster_3,296.933035,2.262368
4,cluster_4,249.520769,-19.275265
5,cluster_5,240.466086,22.570877
6,cluster_6,110.000000,34.000000


In [267]:
custom_cluster_means.to_csv('custom_cluster_means.csv')

In [268]:
cluster_0 = Seymour_Bobby.loc[Seymour_Bobby['scikit_cluster'] == 0]
cluster_1 = Seymour_Bobby.loc[Seymour_Bobby['scikit_cluster'] == 1]
cluster_2 = Seymour_Bobby.loc[Seymour_Bobby['scikit_cluster'] == 2]
cluster_3 = Seymour_Bobby.loc[Seymour_Bobby['scikit_cluster'] == 3]
cluster_4 = Seymour_Bobby.loc[Seymour_Bobby['scikit_cluster'] == 4]
cluster_5 = Seymour_Bobby.loc[Seymour_Bobby['scikit_cluster'] == 5]
#cluster_6 = Seymour_Bobby.loc[Seymour_Bobby['scikit_cluster'] == 6]


scikit_clusters = ['cluster_0','cluster_1','cluster_2','cluster_3','cluster_4','cluster_5','cluster_6']

In [269]:
mean_distances = []
mean_distances.append(cluster_0['distance'].mean())
mean_distances.append(cluster_1['distance'].mean())
mean_distances.append(cluster_2['distance'].mean())
mean_distances.append(cluster_3['distance'].mean())
mean_distances.append(cluster_4['distance'].mean())
mean_distances.append(cluster_5['distance'].mean())
#mean_distances.append(cluster_6['distance'].mean())
mean_distances.append(110)

In [270]:
mean_directions = []
mean_directions.append(cluster_0['direction'].mean())
mean_directions.append(cluster_1['direction'].mean())
mean_directions.append(cluster_2['direction'].mean())
mean_directions.append(cluster_3['direction'].mean())
mean_directions.append(cluster_4['direction'].mean())
mean_directions.append(cluster_5['direction'].mean())
#mean_directions.append(cluster_6['direction'].mean())
mean_directions.append(34)

In [271]:
scikit_cluster_means = pd.DataFrame()
scikit_cluster_means['scikit_cluster'] = scikit_clusters
scikit_cluster_means['distance'] = mean_distances
scikit_cluster_means['direction'] = mean_directions

In [272]:
scikit_cluster_means

,scikit_cluster,distance,direction
0,cluster_0,296.933035,2.262368
1,cluster_1,41.915296,12.329664
2,cluster_2,243.359121,-19.408827
3,cluster_3,240.466086,22.570877
4,cluster_4,35.138833,-8.514702
5,cluster_5,48.328333,-39.419181
6,cluster_6,110.000000,34.000000


In [273]:
data.describe()

,distance,direction
count,250.000000,250.000000
mean,148.759290,2.756925
std,122.354599,16.953629
min,2.137034,-43.084068
25%,18.959499,-8.489706
50%,141.296039,5.284664
75%,263.601156,15.991818
max,373.361555,34.797064


In [274]:
scikit_cluster_means.to_csv('cluster_means.csv')

In [275]:
Seymour_Bobby.to_csv('custom_clusters.csv')

In [276]:
Seymour_Bobby.head()

,Unnamed: 0,batter_name,pitcher_handedness,batter_handedness,inning,outs,strikes,velocity,vertical_release_angle,horizontal_release_angle,...,pfxz,vx0,vy0,vz0,ax,ay,play_result,traditional_cluster,scikit_cluster,custom_cluster
0,300,"Seymour, Bobby",1,1,1.0,1.0,2.0,92.463621,-1.293371,-2.147161,...,8.469302,4.810461,-134.517425,-3.564723,-8.995066,32.286544,1,0.0,2,4.0
1,302,"Seymour, Bobby",1,1,1.0,0.0,1.0,85.221137,0.254052,-3.889359,...,2.617599,8.093095,-123.967974,-0.164367,-15.091854,28.772808,0,4.0,1,1.0
2,424,"Seymour, Bobby",1,1,9.0,0.0,0.0,73.479312,0.594389,-1.295192,...,-1.108555,2.692137,-106.850350,-0.351201,5.834135,20.167107,0,4.0,1,1.0
3,457,"Seymour, Bobby",1,1,3.0,1.0,1.0,75.800159,2.194475,0.608890,...,-7.211672,-0.681799,-110.278789,2.527870,11.848525,19.170036,1,8.0,0,3.0
4,462,"Seymour, Bobby",1,1,3.0,2.0,2.0,89.006952,-1.225558,-4.024021,...,10.753557,8.712936,-129.127566,-3.272390,-13.346503,31.863196,0,8.0,2,4.0


In [277]:
Seymour_Bobby.to_csv('clusters.csv')

In [278]:
data

,distance,direction
0,210.152763,-19.519328
1,4.035207,16.767122
2,3.935966,10.970422
3,309.847859,-8.833638
4,287.297315,-18.036043
...,...,...
245,293.229568,2.609174
246,5.780203,17.874337
247,27.044629,-0.923234
248,249.350455,-23.654236
